In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os
from folium.plugins import HeatMap

In [ ]:
df_2022 = pd.read_csv('../data/raw/feminicide_2022.csv')

,QUAND,LIEU,DÉPARTEMENT,PRÉNOM,ÂGE,N°,CONTENU PUBLICATION,LIEN INSTAGRAM
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22,148,NaN,NaN
1,04/01/2023,Perles,Aisne,–,50,147,"Samedi 7 janvier 2023 à Perles (02), le corps ...",https://www.instagram.com/p/CnSIKbRLwTM/?utm_s...
2,31/12/2022,Charleville-Mézières,Ardennes,–,23,146,"Le samedi 31 décembre, une femme de 23 ans a é...",https://www.instagram.com/p/Cm4bp-qLwox/
3,24/12/2022,Saint-Raphaël,Var,Molka,31,145,"Samedi 24 décembre à Saint-Raphaël (83), une f...",https://www.instagram.com/p/Cm2Jl02LGTZ/
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67,144,"Dimanche 16 octobre à Goudelin (22), une femme...",https://www.instagram.com/p/Cm1kH5gN24G/


In [30]:
df_2022.info()
#mettre en lower le lom des colommess
df_2022.columns = df_2022.columns.str.lower()
#supprimer les espaces dans les noms de colonnes
df_2022.columns = df_2022.columns.str.replace(' ', '_')
#supprmier les caracteres speciaux dans les noms de colonnes
df_2022.columns = df_2022.columns.str.replace(r'[^\w\s]', '', regex=True)
#supprimer les accents dans les noms de colonnes
df_2022.columns = df_2022.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   QUAND                148 non-null    object
 1   LIEU                 148 non-null    object
 2   DÉPARTEMENT          148 non-null    object
 3   PRÉNOM               148 non-null    object
 4   ÂGE                  148 non-null    int64 
 5   N°                   148 non-null    int64 
 6   CONTENU PUBLICATION  147 non-null    object
 7   LIEN INSTAGRAM       147 non-null    object
dtypes: int64(2), object(6)
memory usage: 9.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   quand                148 non-null    object
 1   lieu                 148 non-null    object
 2   departement          148 non-null    object
 3   pr

In [ ]:
#renommer la colonne 'quand' en 'date'
df_2022 = df_2022.rename(columns={'quand': 'date','lieu':'commune'})
#suppression colonnes inutiles
df_2022 = df_2022.drop(columns=['contenu_publication','lien_instagram','n'])


In [35]:
df_2022.head()

,date,commune,departement,prenom,age,n
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22,148
1,04/01/2023,Perles,Aisne,–,50,147
2,31/12/2022,Charleville-Mézières,Ardennes,–,23,146
3,24/12/2022,Saint-Raphaël,Var,Molka,31,145
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67,144


In [43]:
df_2022['commune_sans_accents'] = df_2022['commune'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_2022['departement_sans_accents'] = df_2022['departement'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_2022.head()

,date,commune,departement,prenom,age,n,commune_sans_accents,departement_sans_accents
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22,148,prahecq,deux-sevres
1,04/01/2023,Perles,Aisne,–,50,147,perles,aisne
2,31/12/2022,Charleville-Mézières,Ardennes,–,23,146,charleville-mezieres,ardennes
3,24/12/2022,Saint-Raphaël,Var,Molka,31,145,saint-raphael,var
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67,144,goudelin,cotes-d'armor


In [45]:
#date en format datetime
df_2022['date'] = pd.to_datetime(df_2022['date'], format='%d/%m/%Y', errors='coerce')
df_2022.head()

,date,commune,departement,prenom,age,n,commune_sans_accents,departement_sans_accents
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22,148,prahecq,deux-sevres
1,2023-01-04,Perles,Aisne,–,50,147,perles,aisne
2,2022-12-31,Charleville-Mézières,Ardennes,–,23,146,charleville-mezieres,ardennes
3,2022-12-24,Saint-Raphaël,Var,Molka,31,145,saint-raphael,var
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67,144,goudelin,cotes-d'armor


In [49]:
df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')
df_commune.head()

C:\Users\ghirg\AppData\Local\Temp\ipykernel_17008\954040633.py:1: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')


,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [50]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         34935 non-null  int64  
 1   code_insee                         34935 non-null  object 
 2   nom_standard                       34935 non-null  object 
 3   nom_sans_pronom                    34935 non-null  object 
 4   nom_a                              34935 non-null  object 
 5   nom_de                             34935 non-null  object 
 6   nom_sans_accent                    34935 non-null  object 
 7   nom_standard_majuscule             34935 non-null  object 
 8   typecom                            34935 non-null  object 
 9   typecom_texte                      34935 non-null  object 
 10  reg_code                           34935 non-null  int64  
 11  reg_nom                            34935 non-null  obj

In [53]:
# dans df_commune, récupérer commmune contenant 'Paris'
df_commune[df_commune['nom_standard'].str.contains('Paris', case=False)]

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
14451,14451,38485,Seyssinet-Pariset,Seyssinet-Pariset,à Seyssinet-Pariset,de Seyssinet-Pariset,seyssinet-pariset,SEYSSINET-PARISET,COM,commune,...,5.696,45.173,5.670,1,Grands centres urbains,2.0,centres intermédiaires d'équipements et de ser...,NaN,https://fr.wikipedia.org/wiki/fr:Seyssinet-Par...,https://villedereve.fr/ville/38485-seyssinet-p...
14688,14688,39189,Damparis,Damparis,à Damparis,de Damparis,damparis,DAMPARIS,COM,commune,...,5.412,47.071,5.407,3,Petites villes,1.0,centres locaux d'équipements et de services,Damparisiens,https://fr.wikipedia.org/wiki/fr:Damparis,https://villedereve.fr/ville/39189-damparis
24302,24302,62826,Le Touquet-Paris-Plage,Touquet-Paris-Plage,au Touquet-Paris-Plage,du Touquet-Paris-Plage,le-touquet-paris-plage,LE TOUQUET-PARIS-PLAGE,COM,commune,...,1.587,50.508,1.599,4,Ceintures urbaines,2.0,centres intermédiaires d'équipements et de ser...,Touquettois,https://fr.wikipedia.org/wiki/fr:Le Touquet-Pa...,https://villedereve.fr/ville/62826-le-touquet-...
28102,28102,71343,Paris-l'Hôpital,Paris-l'Hôpital,à Paris-l'Hôpital,de Paris-l'Hôpital,paris-l-hopital,PARIS-L'HÔPITAL,COM,commune,...,4.639,46.913,4.640,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:Paris-l'Hôpital,https://villedereve.fr/ville/71343-paris-l-hop...
29244,29244,75056,Paris,Paris,à Paris,de Paris,paris,PARIS,COM,commune,...,2.352,NaN,NaN,1,Grands centres urbains,4.0,centres majeurs d'équipements et de services,Parisien,https://fr.wikipedia.org/wiki/fr:Paris,https://villedereve.fr/ville/75056-paris
30439,30439,77514,Villeparisis,Villeparisis,à Villeparisis,de Villeparisis,villeparisis,VILLEPARISIS,COM,commune,...,2.617,48.942,2.617,1,Grands centres urbains,3.0,centres structurants d'équipements et de services,Villeparisiens,https://fr.wikipedia.org/wiki/fr:Villeparisis,https://villedereve.fr/ville/77514-villeparisis
31941,31941,81202,Parisot,Parisot,à Parisot,de Parisot,parisot,PARISOT,COM,commune,...,1.831,43.805,1.836,6,Rural à habitat dispersé,1.0,centres locaux d'équipements et de services,Parisotains,https://fr.wikipedia.org/wiki/fr:Parisot (Tarn),https://villedereve.fr/ville/81202-parisot
32197,32197,82137,Parisot,Parisot,à Parisot,de Parisot,parisot,PARISOT,COM,commune,...,1.857,44.262,1.866,7,Rural à habitat très dispersé,1.0,centres locaux d'équipements et de services,Parisotins,https://fr.wikipedia.org/wiki/fr:Parisot (Tarn...,https://villedereve.fr/ville/82137-parisot
34673,34673,95176,Cormeilles-en-Parisis,Cormeilles-en-Parisis,à Cormeilles-en-Parisis,de Cormeilles-en-Parisis,cormeilles-en-parisis,CORMEILLES-EN-PARISIS,COM,commune,...,2.199,48.968,2.199,1,Grands centres urbains,3.0,centres structurants d'équipements et de services,Cormeillais,https://fr.wikipedia.org/wiki/fr:Cormeilles-en...,https://villedereve.fr/ville/95176-cormeilles-...
34689,34689,95241,Fontenay-en-Parisis,Fontenay-en-Parisis,à Fontenay-en-Parisis,de Fontenay-en-Parisis,fontenay-en-parisis,FONTENAY-EN-PARISIS,COM,commune,...,2.451,49.050,2.445,4,Ceintures urbaines,1.0,centres locaux d'équipements et de services,Fontenaysiens,https://fr.wikipedia.org/wiki/fr:Fontenay-en-P...,https://villedereve.fr/ville/95241-fontenay-en...


In [ ]:
#dans df_2022, remplacer les noms de commune contenant '*e arrondissement' par 'Paris'
df_2022['commune_sans_accents'] = df_2022['commune_sans_accents'].str.replace(r'\d{1,2}e arrondissement', 'paris', regex=True)

0                      prahecq
1                       perles
2         charleville-mezieres
3               saint-raphael 
4                     goudelin
                ...           
143                     roujan
144              aubervilliers
145                      nice 
146                      labry
147    bellevigne-les-chateaux
Name: commune_sans_accents, Length: 148, dtype: object

In [57]:
df_2022[df_2022['commune_sans_accents']=='marseille']

,date,commune,departement,prenom,age,n,commune_sans_accents,departement_sans_accents
55,2022-09-07,Marseille,Bouches-du-Rhône,Charlène,36,93,marseille,bouches-du-rhone
56,2022-09-07,Marseille,Bouches-du-Rhône,Marie-Antoinette,70,92,marseille,bouches-du-rhone
114,2022-04-01,Marseille,Bouches-du-Rhône,–,77,34,marseille,bouches-du-rhone


In [69]:
df_commune['nom_sans_accent'] = df_commune['nom_sans_pronom'].str.lower().str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_commune['dep_sans_accents'] = df_commune['dep_nom'].str.lower().str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [62]:
df_commune['nom_sans_accent']

0        abergement-clemenciat
1          abergement-de-varey
2            amberieu-en-bugey
3          amberieux-en-dombes
4                      ambleon
                 ...          
34930            m'tsangamouji
34931                 ouangani
34932                 pamandzi
34933                     sada
34934                 tsingoni
Name: nom_sans_accent, Length: 34935, dtype: object

In [70]:
df_commune.head()

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve,dep_sans_accents
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...,ain
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...,ain
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...,ain
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...,ain
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon,ain


In [71]:
df_commune.columns

Index(['Unnamed: 0', 'code_insee', 'nom_standard', 'nom_sans_pronom', 'nom_a',
       'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'typecom',
       'typecom_texte', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom',
       'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code',
       'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi',
       'code_insee_centre_zone_emploi', 'code_unite_urbaine',
       'nom_unite_urbaine', 'taille_unite_urbaine',
       'type_commune_unite_urbaine', 'statut_commune_unite_urbaine',
       'population', 'superficie_hectare', 'superficie_km2', 'densite',
       'altitude_moyenne', 'altitude_minimale', 'altitude_maximale',
       'latitude_mairie', 'longitude_mairie', 'latitude_centre',
       'longitude_centre', 'grille_densite', 'grille_densite_texte',
       'niveau_equipements_services', 'niveau_equipements_services_texte',
       'gentile', 'url_wikipedia', 'url_villedereve', 'dep_sans_accents'],
      dtype='object')

In [72]:
df_commune = df_commune[['dep_code','population','densite','latitude_mairie','longitude_mairie','nom_sans_accent','dep_sans_accents']]

In [ ]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dep_code          34935 non-null  object 
 1   population        34935 non-null  int64  
 2   densite           34932 non-null  float64
 3   latitude_mairie   34935 non-null  float64
 4   longitude_mairie  34935 non-null  float64
 5   nom_sans_accent   34935 non-null  object 
 6   dep_sans_accents  34935 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 1.9+ MB


In [81]:
#inner join entre df_2022 et df_commune sur les colonnes 'commune_sans_accents' et 'departement_sans_accents' de df_2022 et 'nom_sans_accent' et 'dep_sans_accents' de df_commune
df_merged = pd.merge(df_2022, df_commune, how='left', left_on=['commune_sans_accents'], right_on=['nom_sans_accent'])

In [79]:
df_merged

,date,commune,departement,prenom,age,n,commune_sans_accents,departement_sans_accents,dep_code,population,densite,latitude_mairie,longitude_mairie,nom_sans_accent,dep_sans_accents
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22,148,prahecq,deux-sevres,79,2246.0,90.0,46.259,-0.345,prahecq,deux-sevres
1,2023-01-04,Perles,Aisne,–,50,147,perles,aisne,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-31,Charleville-Mézières,Ardennes,–,23,146,charleville-mezieres,ardennes,08,46398.0,1460.0,49.760,4.720,charleville-mezieres,ardennes
3,2022-12-24,Saint-Raphaël,Var,Molka,31,145,saint-raphael,var,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67,144,goudelin,cotes-d'armor,22,1720.0,74.0,48.605,-3.016,goudelin,cotes-d'armor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2022-01-14,Roujan,Hérault,Caroline,24,5,roujan,herault,34,2260.0,133.0,43.504,3.310,roujan,herault
144,2022-01-08,Aubervilliers,Seine-Saint-Denis,Elsa-Marie,29,4,aubervilliers,seine-saint-denis,93,90071.0,15610.0,48.915,2.382,aubervilliers,seine-saint-denis
145,2022-01-01,Nice,Alpes-Maritimes,Lisa,45,3,nice,alpes-maritimes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2022-01-01,Labry,Meurthe-et-Moselle,Muriel,56,2,labry,meurthe-et-moselle,54,1575.0,254.0,49.173,5.882,labry,meurthe-et-moselle
